In [8]:
# 📦 Install required libraries if not already installed
# pip install pytorch-tabnet imbalanced-learn scikit-learn pandas joblib

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from pytorch_tabnet.tab_model import TabNetClassifier
from imblearn.over_sampling import SMOTE
import torch
import joblib

# ✅ Load dataset
df = pd.read_csv(r""C:\Users\yadav\OneDrive\Documents\NetBeansProjects\FiresaftyProject\src\firesaftyproject\model\tabnet firesafety\fire_safety_dataset.csv"")

# ✅ Drop rows with missing values (if any)
df.dropna(inplace=True)

# ✅ Encode 'building_type'
building_type_encoder = LabelEncoder()
df['building_type'] = building_type_encoder.fit_transform(df['building_type'])

# ✅ Encode target 'status' (Safe=1, Unsafe=0)
status_encoder = LabelEncoder()
df['status'] = status_encoder.fit_transform(df['status'])

# ✅ Separate features and labels
X = df.drop('status', axis=1).values
y = df['status'].values

# ✅ Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# ✅ Balance classes using SMOTE
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# ✅ Normalize features
scaler = StandardScaler()
X_train_res = scaler.fit_transform(X_train_res)
X_test = scaler.transform(X_test)

# ✅ Save encoders and scaler
joblib.dump(building_type_encoder, "building_type_encoder.pkl")
joblib.dump(status_encoder, "status_encoder.pkl")
joblib.dump(scaler, "tabnet_scaler.pkl")

# ✅ Initialize and train TabNet classifier
clf = TabNetClassifier(verbose=1, seed=42)
clf.fit(
    X_train_res, y_train_res,
    eval_set=[(X_test, y_test)],
    eval_name=["test"],
    eval_metric=["accuracy"],
    max_epochs=100,
    patience=10,
    batch_size=256,
    virtual_batch_size=128
)

# ✅ Save TabNet model in .pkl format for integration with IntelliJ
joblib.dump(clf, "tabnet_model.pkl")

# ✅ Evaluate
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("\n📈 Classification Report:")
print(classification_report(y_test, y_pred, target_names=status_encoder.classes_))
print(f"\n✅ Accuracy: {acc * 100:.2f}%")


C:\Users\Nishita\AppData\Roaming\Python\Python311\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.74742 | test_accuracy: 0.39    |  0:00:00s
epoch 1  | loss: 0.68702 | test_accuracy: 0.555   |  0:00:00s
epoch 2  | loss: 0.68056 | test_accuracy: 0.49    |  0:00:01s
epoch 3  | loss: 0.67471 | test_accuracy: 0.45    |  0:00:01s
epoch 4  | loss: 0.66681 | test_accuracy: 0.5625  |  0:00:01s
epoch 5  | loss: 0.65854 | test_accuracy: 0.57    |  0:00:02s
epoch 6  | loss: 0.6603  | test_accuracy: 0.565   |  0:00:02s
epoch 7  | loss: 0.6493  | test_accuracy: 0.4925  |  0:00:02s
epoch 8  | loss: 0.64894 | test_accuracy: 0.5475  |  0:00:02s
epoch 9  | loss: 0.64557 | test_accuracy: 0.4975  |  0:00:03s
epoch 10 | loss: 0.64004 | test_accuracy: 0.4825  |  0:00:03s
epoch 11 | loss: 0.64687 | test_accuracy: 0.515   |  0:00:03s
epoch 12 | loss: 0.64736 | test_accuracy: 0.535   |  0:00:04s
epoch 13 | loss: 0.64551 | test_accuracy: 0.4825  |  0:00:04s
epoch 14 | loss: 0.64547 | test_accuracy: 0.495   |  0:00:04s
epoch 15 | loss: 0.64417 | test_accuracy: 0.5275  |  0:00:05s

Early s

C:\Users\Nishita\AppData\Roaming\Python\Python311\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



📈 Classification Report:
              precision    recall  f1-score   support

        safe       0.76      0.63      0.69       302
      unsafe       0.25      0.39      0.31        98

    accuracy                           0.57       400
   macro avg       0.51      0.51      0.50       400
weighted avg       0.64      0.57      0.59       400


✅ Accuracy: 57.00%


In [14]:
import joblib
import numpy as np

# ✅ Load saved model and preprocessing tools
clf = joblib.load("tabnet_model.pkl")
building_type_encoder = joblib.load("building_type_encoder.pkl")
status_encoder = joblib.load("status_encoder.pkl")
scaler = joblib.load("tabnet_scaler.pkl")

# ✅ Example: User input (replace with actual input or UI values)
# Sample values: ["Residential", 5 floors, 20 rooms, 150 capacity, 3000 area, 5 extinguishers, 2 alarms]
user_input_raw = {
    "building_type": "educational",
    "floors": 30,
    "rooms": 198,
    "capacity": 231,
    "area": 29645,
    "extinguishers": 61,
    "sprinklers": 100,
    "smoke_detectors": 99,
    "fire_alarms": 11
}

# ✅ Convert input into feature array
user_building_type_encoded = building_type_encoder.transform([user_input_raw["building_type"]])[0]

user_features = np.array([[
    user_building_type_encoded,
    user_input_raw["floors"],
    user_input_raw["rooms"],
    user_input_raw["capacity"],
    user_input_raw["area"],
    user_input_raw["extinguishers"],
    user_input_raw["sprinklers"],
    user_input_raw["smoke_detectors"],
    user_input_raw["fire_alarms"]
]])

# ✅ Scale the input features
user_features_scaled = scaler.transform(user_features)

# ✅ Predict using the model
prediction = clf.predict(user_features_scaled)[0]
predicted_label = status_encoder.inverse_transform([prediction])[0]

# ✅ Output result
if predicted_label.lower() == "unsafe":
    print("❌ Provided data is not as per the fire safety guidelines. Application rejected.")
else:
    print("✅ Application accepted for further verification. Data meets fire safety requirements.")


✅ Application accepted for further verification. Data meets fire safety requirements.
